In [ ]:
import numpy as np
import cv2 as cv2
import tensorflow as tf

from random import shuffle

from tqdm import tqdm
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

Pre-Processing IMG

In [5]:
#travel to dir of image
width = 224
num_class = 2

trainpath = 'badSizeTest/labelData/'
#testpath = 'Mold/TrainMold/'


trainImg = [trainpath + f for f in os.listdir(trainpath)]
#testImg = [testpath + f for f in os.listdir(testpath)]

shuffle(trainImg)

print(trainImg)

['Size/TriainSize/Ok(473).JPG', 'Size/TriainSize/Ok(224).JPG', 'Size/TriainSize/Bad(31).JPG', 'Size/TriainSize/Ok(39).JPG', 'Size/TriainSize/Ok(81).JPG', 'Size/TriainSize/Ok(146).JPG', 'Size/TriainSize/Ok(910).JPG', 'Size/TriainSize/Ok(891).JPG', 'Size/TriainSize/Ok(722).JPG', 'Size/TriainSize/Ok(211).JPG', 'Size/TriainSize/Ok(1306).JPG', 'Size/TriainSize/Ok(713).JPG', 'Size/TriainSize/Ok(1508).JPG', 'Size/TriainSize/Ok(1634).JPG', 'Size/TriainSize/Ok(1437).JPG', 'Size/TriainSize/Ok(796).JPG', 'Size/TriainSize/Ok(1279).JPG', 'Size/TriainSize/Ok(1342).JPG', 'Size/TriainSize/Ok(1027).JPG', 'Size/TriainSize/Bad(1250).JPG', 'Size/TriainSize/Bad(899).JPG', 'Size/TriainSize/Ok(144).JPG', 'Size/TriainSize/Bad(1757).JPG', 'Size/TriainSize/Bad(1409).JPG', 'Size/TriainSize/Bad(298).JPG', 'Size/TriainSize/Ok(1730).JPG', 'Size/TriainSize/Bad(692).JPG', 'Size/TriainSize/Ok(1162).JPG', 'Size/TriainSize/Bad(1325).JPG', 'Size/TriainSize/Ok(1226).JPG', 'Size/TriainSize/Bad(1384).JPG', 'Size/TriainSize/

In [6]:
#to label to answer of each img (function)
def getlabel(imgName):
   l = imgName.split('/')[2]
   l = l.split("(")[0]
   #
   if(l == 'Ok'):
     return(np.array([1,0]))
   elif(l == 'Bad'):
     return(np.array([0,1]))


In [7]:
#read img file and append in array (function)
def img2data(path):
  trainData = []
  for item in tqdm((path)):
      img = cv2.imread(item)
      img = cv2.resize(img, (width,width))

      trainData.append([np.array(img), getlabel(item)])
      #plt.imshow(img)
      #plt.show()
  return trainData


In [8]:
#read img file and append in array
trainSet = img2data(trainImg)

100%|██████████| 3688/3688 [00:09<00:00, 397.88it/s]


In [9]:
#Split Test and Train dataset
train = trainSet[:-700]
test = trainSet[-700:]

print(len(train))
print(len(test))

xtrain = np.array([i[0] for i in train]).reshape(-1, width, width, 3)/255
ytrain = np.array([i[1] for i in train])

xtest = np.array([i[0] for i in test]).reshape(-1, width, width, 3)/255
ytest = np.array([i[1] for i in test])

2988
700


*End PreProcess IMG*

*MobileNetV2*

In [10]:
baseMobileNetV2Model = tf.keras.applications.MobileNetV2(input_shape = (224,224,3), include_top = False, weights="imagenet")
baseMobileNetV2Model.trainable = False

avgPoolingLayer =  tf.keras.layers.GlobalAveragePooling2D()(baseMobileNetV2Model.output)
predLayer = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayer)

MobileNetV2model = tf.keras.models.Model(inputs = baseMobileNetV2Model.input, outputs = predLayer)

In [11]:
MobileNetV2model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
MobileNetV2model.fit(xtrain, ytrain, epochs=2 , validation_data = (xtest,ytest), batch_size = 100)

Epoch 1/2
30/30 [==============================] - 102s 3s/step - loss: 0.6042 - accuracy: 0.6717 - val_loss: 0.5776 - val_accuracy: 0.6900
Epoch 2/2
30/30 [==============================] - 99s 3s/step - loss: 0.5683 - accuracy: 0.7095 - val_loss: 0.5470 - val_accuracy: 0.7386


In [12]:
pred = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(MobileNetV2model.predict(rimg))
   if result == 0:
      pred.append([1,0])
   if result == 1:
      pred.append([0,1])


print(classification_report(pred, ytest))



1/1 [==============================] - 0s 53ms/step
              precision    recall  f1-score   support

           0       0.79      0.73      0.76       393
           1       0.68      0.75      0.72       307

   micro avg       0.74      0.74      0.74       700
   macro avg       0.74      0.74      0.74       700
weighted avg       0.74      0.74      0.74       700
 samples avg       0.74      0.74      0.74       700



*Finished*

DenseNet

In [13]:
baseDenseModel = tf.keras.applications.DenseNet121(
   input_shape = (224,224,3),
   include_top=False,
   weights="imagenet",
)

baseDenseModel.trainable = False

In [14]:
avgPoolingLayerDen =  tf.keras.layers.GlobalAveragePooling2D()(baseDenseModel.output)
predLayerDen = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayerDen)

DenseNetmodel = tf.keras.models.Model(inputs = baseDenseModel.input, outputs = predLayerDen)

In [15]:
DenseNetmodel.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
DenseNetmodel.fit(xtrain, ytrain, epochs=2, validation_data = (xtest,ytest), batch_size = 100)

Epoch 1/2
30/30 [==============================] - 418s 14s/step - loss: 0.9306 - accuracy: 0.5007 - val_loss: 0.7807 - val_accuracy: 0.4829
Epoch 2/2
30/30 [==============================] - 412s 14s/step - loss: 0.7565 - accuracy: 0.4993 - val_loss: 0.7146 - val_accuracy: 0.5271


In [16]:
predDen = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(DenseNetmodel.predict(rimg))
   if result == 0:
      predDen.append([1,0])
   if result == 1:
      predDen.append([0,1])

print(classification_report(predDen, ytest))

1/1 [==============================] - 0s 228ms/step
              precision    recall  f1-score   support

           0       0.55      0.55      0.55       365
           1       0.51      0.51      0.51       335

   micro avg       0.53      0.53      0.53       700
   macro avg       0.53      0.53      0.53       700
weighted avg       0.53      0.53      0.53       700
 samples avg       0.53      0.53      0.53       700



*Finished*

Inception

In [17]:
baseInceptionModel = tf.keras.applications.InceptionV3(
   input_shape = (224,224,3),
   include_top=False,
   weights="imagenet",
)

baseInceptionModel.trainable = False

In [18]:
avgPoolingLayerIn =  tf.keras.layers.GlobalAveragePooling2D()(baseInceptionModel.output)
predLayerIn = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayerIn)

Inceptionmodel = tf.keras.models.Model(inputs = baseInceptionModel.input, outputs = predLayerIn)

In [19]:
Inceptionmodel.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
Inceptionmodel.fit(xtrain, ytrain, epochs=2, validation_data = (xtest,ytest), batch_size = 100)

Epoch 1/2
30/30 [==============================] - 255s 8s/step - loss: 0.8567 - accuracy: 0.4602 - val_loss: 0.7137 - val_accuracy: 0.5557
Epoch 2/2
30/30 [==============================] - 192s 6s/step - loss: 0.6660 - accuracy: 0.6218 - val_loss: 0.6126 - val_accuracy: 0.6829


In [20]:
predIn = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(Inceptionmodel.predict(rimg))
   if result == 0:
      predIn.append([1,0])
   if result == 1:
      predIn.append([0,1])

print(classification_report(predIn, ytest))

1/1 [==============================] - 0s 117ms/step
              precision    recall  f1-score   support

           0       0.74      0.68      0.71       394
           1       0.62      0.69      0.65       306

   micro avg       0.68      0.68      0.68       700
   macro avg       0.68      0.68      0.68       700
weighted avg       0.69      0.68      0.68       700
 samples avg       0.68      0.68      0.68       700



*Finished*

Xception

In [21]:
baseXceptionmodel = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    classifier_activation="softmax",
)

baseXceptionmodel.trainable = False

In [22]:
avgPoolingLayer3 =  tf.keras.layers.GlobalAveragePooling2D()(baseXceptionmodel.output)
predLayer3 = tf.keras.layers.Dense(units=2, activation='softmax')(avgPoolingLayer3)

Xceptionmodel = tf.keras.models.Model(inputs = baseXceptionmodel.input, outputs = predLayer3)

In [23]:
Xceptionmodel.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics = ["accuracy"])
Xceptionmodel.fit(xtrain, ytrain, epochs=2, validation_data = (xtest,ytest), batch_size = 100)

Epoch 1/2
19/30 [==================>...........] - ETA: 1:47 - loss: 0.6904 - accuracy: 0.5511

In [ ]:
pred3 = []
for img in xtest:
   rimg = np.reshape(img, (-1,224,224,3))
   result = np.argmax(Xceptionmodel.predict(rimg))
   if result == 0:
      pred3.append([1,0])
   if result == 1:
      pred3.append([0,1])
   

#print(pred3)
#print(ytest)

print(classification_report(pred3, ytest))

1/1 [==============================] - 0s 173ms/step
              precision    recall  f1-score   support

           0       0.75      0.69      0.72       392
           1       0.64      0.71      0.67       308

   micro avg       0.70      0.70      0.70       700
   macro avg       0.70      0.70      0.70       700
weighted avg       0.70      0.70      0.70       700
 samples avg       0.70      0.70      0.70       700



*Finished*

Summary all

In [ ]:
print("MobileNetV2 report")
print(classification_report(pred, ytest))
print(confusion_matrix(pred, ytest))

print("########################################")

print("DenseNet121 report")
print(classification_report(predDen, ytest))
print(confusion_matrix(predDen, ytest))

print("########################################")

print("Incception report")
print(classification_report(predIn, ytest))
print(confusion_matrix(predIn, ytest))

print("########################################")

print("Xception report")
print(classification_report(pred3, ytest))
print(confusion_matrix(pred3, ytest))

print("########################################")

MobileNetV2 report


NameError: name 'pred' is not defined